In [ ]:
#if the notebook is opened in Google Colab, this cell may be used. 
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/SYSC4906/
!git clone https://github.com/MichaelPatsula/SYSC4906-Project.git
%cd SYSC4906-Project/
!git pull

In [1]:
#May be needed to get the most recent plotly dependency
%pip install plotly --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install adjustText

Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install dash

In [60]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import json
import plotly.express as px
import plotly.io as pio
from wordcloud import WordCloud, STOPWORDS
from adjustText import adjust_text
import ipywidgets as widgets

import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc

Set to 'colab' if using Google Colab, or set to 'notebook' if using Jupyter notebook

In [43]:
pio.renderers.default = 'notebook'

In [44]:
reverse_map = {'Abbotsford': 'Abbotsford, BC',
         'Barrie': 'Barrie, ON',
         'Brampton': "Brampton, ON",
         'Burlington': 'Burlington, ON',
         'Burnaby': 'Burnaby, BC',
         'Calgary': 'Calgary, AB',
         'Coquitlam': 'Coquitlam, BC',
         'Edmonton': 'Edmonton, AB',
         'Fredericton': 'Fredericton, NB',
         'Guelph': 'Guelph, ON',
         'Halifax': 'Halifax, NS',
         'Hamilton': 'Hamilton, ON',
         'Kamloops': 'Kamloops, BC',
         'Kelowna': 'Kelowna, BC',
         'Kingston': 'Kingston, ON',
         'Kitchener': 'Kitchener, ON',
         'Langley, BC': 'Langley, BC',
         'Lethbridge, AB': 'Lethbridge, AB',
         'London':'London, ON',
         'Markham': 'Markham ON',
         'Mississauga': 'Mississauga, ON',
         'Moncton': 'Moncton, NB',
         'Montreal': 'Montreal, QC',
         'Nanaimo, BC': 'Nanaimo, BC',
         'New Westminster': 'New Westminster, BC',
         'North Vancouver': 'North Vancouver, BC',
         'Oakville': 'Oakville, ON',
         'Oshawa': 'Oshawa, ON',
         'Ottawa': 'Ottawa, ON',
         'Quebec City': 'Quebec City, QC',
         'Red Deer': 'Red Deer, AB',
         'Regina': 'Regina, SK',
         'Richmond Hill': 'Richmond Hill, ON',
         "Saint John's": "Saint John's, NL",
         'Saskatoon': 'Saskatoon, SK',
         'St.Catharines': 'St.Catharines, ON',
         'Sudbury': 'Sudbury, ON',
         'Surrey': 'Surrey, BC',
         'Thunder Bay': 'Thunder Bay, ON',
         'Toronto': 'Toronto, ON',
         'Vancouver':'Vancouver, BC',
         'Vaughan': 'Vaughan, ON',
         'Victoria': 'Victoria, BC',
         'Waterloo': 'Waterloo, ON',
         'Windsor': 'Windsor, ON',
         'Winnipeg': 'Winnipeg, MB'
}

**Step #2**: Extract data from .csv and into dataframes

In [45]:
jobs = pd.read_csv("DataSet/JobPostDetails/data.csv")
noc_element = pd.read_csv("DataSet/NOC/noc-cnp-2016-element-v4-eng.csv")
noc_structure = pd.read_csv("DataSet/NOC/noc-cnp-2016-structure-v4-eng.csv")
job_levels = pd.read_csv("DataSet/NOC/JobLevels.csv")

#For each file in the Numbeo Data directory insert a list of the data into the numbeo map
numbeo = {}
for dataset in os.listdir("DataSet/Numbeo Data"):
    data = dataset.split(".")[0]
    numbeo[data] = pd.read_csv("DataSet/Numbeo Data/" + dataset)
    numbeo[data].drop("Rank", axis = 1, inplace=True)
    numbeo[data]["City"] = numbeo[data]["City"].apply(lambda x: reverse_map[x])

**Step #3:** Clean the noc_structure and the job_levels dataframe and then marge the two dataframes. 

In [46]:
#Filter the noc_structure to only Unit group and select relevant columns
noc_structure = noc_structure[noc_structure["Hierarchical structure"] == "Unit group"]
noc_structure["Code"] = noc_structure["Code"].astype(np.float64)
noc_structure = noc_structure[["Code", "Class title", "Class definition"]]

#Convert NocNo from job levels to float to merge**Step #4:** Clean the job dataframe by filtering to only hour data and taking the average pay wjem available otherwise take the min 
job_levels["NocNo"] = job_levels["NocNo"].astype(np.float64)

#Merge the NOC information with jobs
noc_info = pd.merge(noc_structure, job_levels, left_on="Code", right_on="NocNo", how="inner")
noc_info = noc_info.drop("Code", axis=1)

**Step #4:** Clean the job dataframe by filtering to only hour data and taking the average pay wjem available otherwise take the min 

In [47]:
#Clean all jobs to contian only hourly data
jobs_cleaned = jobs.copy(deep=True)
jobs_cleaned = jobs_cleaned[jobs_cleaned["PayUnit"] == "HOUR"]

#Remove some incorrectly labelled hourly data using a heuristic 
jobs_cleaned = jobs_cleaned[~jobs_cleaned["MinPay"].str.contains(',').fillna(False)]
jobs_cleaned = jobs_cleaned[~jobs_cleaned["MaxPay"].str.contains(',').fillna(False)]
jobs_cleaned["MinPay"] = jobs_cleaned["MinPay"].astype(np.float64)
jobs_cleaned["MaxPay"] = jobs_cleaned["MaxPay"].astype(np.float64)

#Get average pay if it exists else use min pay
def get_average_pay(row):
  if np.isnan(row["MaxPay"]):
    return row["MinPay"]
  else:
    return (row["MinPay"] + row["MaxPay"])/2

jobs_cleaned["AveragePay"] = jobs_cleaned.apply(get_average_pay, axis=1)
jobs_cleaned["City"] = jobs_cleaned["City"].str.strip("based in ")

jobs_noc_merged = pd.merge(noc_info, jobs_cleaned, on="NocNo", how="inner")

In [48]:
#These are the numbeo datasets
numbeo.keys()

dict_keys(['CostofLiving', 'Crime', 'HealthCare', 'PropertyPrices', 'Quality of Life', 'Traffic'])

In [54]:
def merge_numbeo_jobs(onDataset):
  return pd.merge(jobs_noc_merged.copy(deep=True), numbeo[onDataset], on="City", how="inner")


In [59]:
def get_merged_kpi_df(numbeoKey):
    pay_merged = merge_numbeo_jobs(numbeoKey)
    #Consider only hourly
    pay_merged = pay_merged[pay_merged["PayUnit"] == "HOUR"]

    #Create a map of each city and its corresponding average pay.
    city_pay = {"City":[], "Average Pay":[]}
    for city, group in pay_merged.groupby("City"):
        group["MinPay"] = group["MinPay"].astype(np.float64)
        city_pay["City"].append(city)
        city_pay["Average Pay"].append(group["MinPay"].mean())
    city_pay = pd.DataFrame(city_pay)

    #Merge the city_pay dataframe with the Numbeo dataset values to get access to the KPIs for each city
    answer_dataframe = pd.merge(city_pay, numbeo[numbeoKey], on="City", how="inner")
    return answer_dataframe

In [61]:
def graphScatter2(independent_col, dependent_col, title, df):
    fig = px.scatter(df, x=independent_col, y=dependent_col, trendline="ols", text="City", title=title, width=1000, height=800)
    return fig;
    #fig.show()

independent_column = "Affordability Index"
dependent_column = "Average Pay"

graphScatter2(independent_column, dependent_column, "Affordability Index vs Average Pay",  get_merged_kpi_df('PropertyPrices'))



In [74]:
def get_dropdown_options(numbeoKey):
    df = get_merged_kpi_df(numbeoKey)
    df = df.drop(columns=['City'])
    col_options = [dict(label=x, value=x) for x in df.columns]
    return col_options

[{'label': 'Average Pay', 'value': 'Average Pay'},
 {'label': 'Price To Income Ratio', 'value': 'Price To Income Ratio'},
 {'label': 'Gross Rental Yield City Centre',
  'value': 'Gross Rental Yield City Centre'},
 {'label': 'Gross Rental Yield Outside of Centre',
  'value': 'Gross Rental Yield Outside of Centre'},
 {'label': 'Price To Rent Ratio City Centre',
  'value': 'Price To Rent Ratio City Centre'},
 {'label': 'Price To Rent Ratio Outside Of City Centre',
  'value': 'Price To Rent Ratio Outside Of City Centre'},
 {'label': 'Mortgage As A Percentage Of Income',
  'value': 'Mortgage As A Percentage Of Income'},
 {'label': 'Affordability Index', 'value': 'Affordability Index'}]

In [ ]:
from dash.dependencies import State, Input, Output

app = dash.Dash(external_stylesheets=[dbc.themes.SUPERHERO])

### defining the HTML component
app.layout = dbc.Container(
    html.Div(
    children=[
                html.H1("Relationship of Professional Industry and Salary to Regional KPIs"),
        
                html.H3("1. Average Hourly Pay to various Property Prices KPI:"),
                html.Div(dcc.Dropdown(id="property_drop_down", options=get_dropdown_options("PropertyPrices"),
                                      value='Affordability Index'
                                     ), style= {
                                              "color": "black",
                                              "text-align": "center",
                                              "background-color": "white",
                                              "border-style": "solid", 
                                              "display": "inline-block", 
                                              "width": "40%"
                                              }), 
                html.Div(dcc.Graph(id ="property_plot")),
        
                html.H3("2. Average Hourly Pay to various Quality of Life KPI:"),
                html.Div(dcc.Dropdown(id="qol_drop_down", options=get_dropdown_options("Quality of Life"),
                                      value='Quality of Life Index'
                                     ), style= {
                                              "color": "black",
                                              "text-align": "center",
                                              "background-color": "white",
                                              "border-style": "solid", 
                                              "display": "inline-block", 
                                              "width": "40%"
                                              }), 
                html.Div(dcc.Graph(id ="qol_plot")), 
        
                html.H3("3. Average Hourly Pay to various Traffic KPI:"),
                html.Div(dcc.Dropdown(id="traffic_drop_down", options=get_dropdown_options("Traffic"),
                                      value='Traffic Index'
                                     ), style= {
                                              "color": "black",
                                              "text-align": "center",
                                              "background-color": "white",
                                              "border-style": "solid", 
                                              "display": "inline-block", 
                                              "width": "40%"
                                              }), 
                html.Div(dcc.Graph(id ="traffic_plot")),
        
                html.H3("4. Average Hourly Pay to various Cost of Living KPI:"),
                html.Div(dcc.Dropdown(id="col_drop_down", options=get_dropdown_options("CostofLiving"),
                                      value='Cost of Living Index'
                                     ), style= {
                                              "color": "black",
                                              "text-align": "center",
                                              "background-color": "white",
                                              "border-style": "solid", 
                                              "display": "inline-block", 
                                              "width": "40%"
                                              }), 
                html.Div(dcc.Graph(id ="col_plot")),
        
                html.H3("Cities with highest hourly wage for a specified profession:"),
                html.P("For multiple keywords, add a comma. (e.g. computer,web,software)"),
                dbc.Input(id="job_title_input", type="text", placeholder="Enter a job title keyword"),
                html.Div(dcc.Graph(id ="plot_area2")),
                                
            ]
))
                               
### Event handling ###

# Property Plot
@app.callback(Output("property_plot","figure"),
              Input("property_drop_down", "value")
)         
def update_property_plot(input_cat):
    df = get_merged_kpi_df('PropertyPrices')
    plot = px.scatter(df, x=input_cat, y="Average Pay", trendline="ols", text="City", title=f"{input_cat} vs Average Pay" , width=1000, height=800)
    return plot.update_layout(title_x=0.5)

# Quality of Life Plot
@app.callback(Output("qol_plot","figure"),
              Input("qol_drop_down", "value")
)             
def update_qol_plot(input_cat):
    df = get_merged_kpi_df('Quality of Life')
    plot = px.scatter(df, x=input_cat, y="Average Pay", trendline="ols", text="City", title=f"{input_cat} vs Average Pay" , width=1000, height=800)
    return plot.update_layout(title_x=0.5)

# Traffic Plot
@app.callback(Output("traffic_plot","figure"),
              Input("traffic_drop_down", "value")
)             
def update_traffic_plot(input_cat):
    df = get_merged_kpi_df('Traffic')
    plot = px.scatter(df, x=input_cat, y="Average Pay", trendline="ols", text="City", title=f"{input_cat} vs Average Pay" , width=1000, height=800)
    return plot.update_layout(title_x=0.5)

# Cost of Living Plot
@app.callback(Output("col_plot","figure"),
              Input("col_drop_down", "value")
)             
def update_col_plot(input_cat):
    df = get_merged_kpi_df('CostofLiving')
    plot = px.scatter(df, x=input_cat, y="Average Pay", trendline="ols", text="City", title=f"{input_cat} vs Average Pay" , width=1000, height=800)
    return plot.update_layout(title_x=0.5)

# Highest wage for profession
@app.callback(
    Output("plot_area2", "figure"),
    Input("job_title_input", "value"),
)
def update_output(job_title_input):
    job_title_input = job_title_input.lower()
    keywords = job_title_input.split(",")
    
    val = jobs_noc_merged["Job Title"].str.lower().str.contains("|".join(keywords))
    selected = jobs_noc_merged[val].copy(deep=True)
    selected = selected[selected["AveragePay"]<80]

    selected = selected.groupby("City", as_index=False).mean()
    selected = selected.sort_values("AveragePay", ascending=False)

    selected_plot = selected.head(20)
    
    plot =  px.bar(selected_plot, x="City", y='AveragePay', title=f"Highest hourly Wage for jobs with keywords: [{job_title_input}] per City")
    return plot.update_layout(title_x=0.5)

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2021 18:04:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2021 18:04:00] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2021 18:04:00] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2021 18:04:00] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:00] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py

ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\christophe\appdata\local\programs\python\python38\lib\site-packag

127.0.0.1 - - [09/May/2021 18:04:14] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2021 18:04:14] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2021 18:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2021 18:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
